In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="sDfIWRbiccdW3QpuRqIs")
project = rf.workspace("bethaniaworkspace").project("motorcycledetection-plboa")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [2]:
import os

print("Dataset location:", dataset.location)
print("Folders inside:")
print(os.listdir(dataset.location))

Dataset location: C:\Users\samue\MotorcycleDetection-2
Folders inside:
['data.yaml', 'data1.yaml', 'merged_all', 'README.dataset.txt', 'README.roboflow.txt', 'runs', 'test', 'train', 'val', 'yolo11n.pt', 'yolov8n.pt']


In [3]:
import os
import yaml

# Path ke file data.yaml yang diunduh
data_yaml_path = os.path.join(dataset.location, "data.yaml")

# Baca isinya
with open(data_yaml_path, "r") as f:
    data = yaml.safe_load(f)

print(data)


{'names': ['motorBebek', 'motorListrik', 'motorMatic', 'motorSport', 'motorTrail'], 'nc': 5, 'roboflow': {'license': 'CC BY 4.0', 'project': 'motorcycledetection-plboa', 'url': 'https://universe.roboflow.com/bethaniaworkspace/motorcycledetection-plboa/dataset/2', 'version': 2, 'workspace': 'bethaniaworkspace'}, 'test': 'test/images', 'train': 'train/images', 'val': 'val/images', 'valid': 'valid/images'}


In [4]:
print(dataset.location)


C:\Users\samue\MotorcycleDetection-2


In [5]:
import yaml

with open(os.path.join(dataset.location, "data.yaml")) as f:
    data = yaml.safe_load(f)
    print("Classes from YAML:", data["names"])

Classes from YAML: ['motorBebek', 'motorListrik', 'motorMatic', 'motorSport', 'motorTrail']


In [6]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [7]:
path_to_project='C:/Users/samue/MotorcycleDetection-2'

os.chdir(path_to_project)

# Verifikasi bahwa Anda berada di direktori yang benar
print(os.getcwd())

C:\Users\samue\MotorcycleDetection-2


In [8]:
data = {
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images'
}

# Simulasikan objek 'dataset' yang memiliki atribut 'location'
class Dataset:
    # Corrected typo: __init__ instead of _init_
    def __init__(self, location):
        self.location = location

# Now this line will correctly call the __init__ method
dataset = Dataset(path_to_project)

# --- Langkah 3: Jalankan Kode Anda ---

splits = ['train', 'val', 'test']
split_counts = {}

print(f"\nProcessing splits for dataset at: {dataset.location}")

for split in splits:
    split_path_relative = data.get(split) # Contoh: 'train/images'
    if split_path_relative:
        # full_path akan menjadi '/content/drive/MyDrive/CV-Thania/MotorcycleDetection-2/train/images'
        full_path = os.path.join(dataset.location, split_path_relative)

        # labels_dir akan menjadi '/content/drive/MyDrive/CV-Thania/MotorcycleDetection-2/train/labels'
        labels_dir = full_path.replace("images", "labels")

        print(f"Checking labels directory for '{split}': {labels_dir}")

        if os.path.exists(labels_dir):
            # Hitung file .txt di dalam folder labels
            num_files = len([f for f in os.listdir(labels_dir) if f.endswith(".txt")])
            split_counts[split] = num_files
            print(f"  Found {num_files} .txt files.")
        else:
            split_counts[split] = 0
            print(f"  Labels directory not found.")
    else:
        split_counts[split] = 0
        print(f"  Relative path for '{split}' not found in 'data' dictionary.")

# --- Langkah 4: Cetak Hasil ---
print("\nJumlah Data Tiap Split:")
for split in splits:
    print(f"{split}: {split_counts[split]}")


Processing splits for dataset at: C:/Users/samue/MotorcycleDetection-2
Checking labels directory for 'train': C:/Users/samue/MotorcycleDetection-2\train/labels
  Found 1052 .txt files.
Checking labels directory for 'val': C:/Users/samue/MotorcycleDetection-2\val/labels
  Found 219 .txt files.
Checking labels directory for 'test': C:/Users/samue/MotorcycleDetection-2\test/labels
  Found 110 .txt files.

Jumlah Data Tiap Split:
train: 1052
val: 219
test: 110


In [9]:
import os
import random
import shutil

# Path asal
image_dir = 'C:/Users/samue/MotorcycleDetection-2/merged_all/images'
label_dir = 'C:/Users/samue/MotorcycleDetection-2/merged_all/labels'

# Path tujuan (train/val)
output_base = 'C:/Users/samue/MotorcycleDetection-2/merged_all'
train_img_dir = os.path.join(output_base, 'images/train')
val_img_dir = os.path.join(output_base, 'images/val')
train_lbl_dir = os.path.join(output_base, 'labels/train')
val_lbl_dir = os.path.join(output_base, 'labels/val')

# Membuat folder jika belum ada
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(train_lbl_dir, exist_ok=True)
os.makedirs(val_lbl_dir, exist_ok=True)

# Ambil semua file gambar
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Shuffle dan split
random.seed(42)
random.shuffle(image_files)
split_ratio = 0.8
split_idx = int(len(image_files) * split_ratio)

train_files = image_files[:split_idx]
val_files = image_files[split_idx:]

# Fungsi untuk menyalin gambar dan label
def copy_data(file_list, img_target, lbl_target):
    for img_file in file_list:
        label_file = os.path.splitext(img_file)[0] + '.txt'
        img_src = os.path.join(image_dir, img_file)
        lbl_src = os.path.join(label_dir, label_file)

        img_dst = os.path.join(img_target, img_file)
        lbl_dst = os.path.join(lbl_target, label_file)

        if os.path.exists(lbl_src):
            shutil.copy2(img_src, img_dst)
            shutil.copy2(lbl_src, lbl_dst)

# Salin file ke train dan val
copy_data(train_files, train_img_dir, train_lbl_dir)
copy_data(val_files, val_img_dir, val_lbl_dir)

print(f"✅ Split selesai. Total: {len(image_files)} gambar | Train: {len(train_files)} | Val: {len(val_files)}")


✅ Split selesai. Total: 1381 gambar | Train: 1104 | Val: 277


In [10]:
import os, shutil

# Path dasar dataset
base_path = "C:/Users/samue/MotorcycleDetection-2"
splits = ["train", "valid", "test"]
merged_images_dir = os.path.join(base_path, "merged_all", "images")
merged_labels_dir = os.path.join(base_path, "merged_all", "labels")

# Buat folder gabungan jika belum ada
os.makedirs(merged_images_dir, exist_ok=True)
os.makedirs(merged_labels_dir, exist_ok=True)

# Gabungkan semua file dari split ke merged_all
for split in splits:
    split_image_dir = os.path.join(base_path, split, "images")
    split_label_dir = os.path.join(base_path, split, "labels")

    if not os.path.exists(split_image_dir) or not os.path.exists(split_label_dir):
        print(f"Skipping missing split: {split}")
        continue

    for img_file in os.listdir(split_image_dir):
        src = os.path.join(split_image_dir, img_file)
        dst = os.path.join(merged_images_dir, img_file)
        shutil.copy2(src, dst)

    for label_file in os.listdir(split_label_dir):
        src = os.path.join(split_label_dir, label_file)
        dst = os.path.join(merged_labels_dir, label_file)
        shutil.copy2(src, dst)

# Cek hasil
num_images = len(os.listdir(merged_images_dir))
num_labels = len(os.listdir(merged_labels_dir))

print("✅ Penggabungan Selesai")
print(f"Total Images (gabungan): {num_images}")
print(f"Total Labels (gabungan): {num_labels}")


Skipping missing split: valid
✅ Penggabungan Selesai
Total Images (gabungan): 1383
Total Labels (gabungan): 1383


In [11]:
# !cat "/content/drive/MyDrive/Semester 8/Praktikum Computer Vision/Project UTS/MotorcycleDetection-2/data.yaml"


In [12]:
import torch
torch.cuda.empty_cache()


In [13]:
import torch
import torchvision

print("CUDA Available:", torch.cuda.is_available())
print("Torch Version:", torch.__version__)
print("TorchVision Version:", torchvision.__version__)
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")


CUDA Available: True
Torch Version: 2.7.1+cu118
TorchVision Version: 0.22.1+cu118
Device: NVIDIA GeForce RTX 3050 Laptop GPU


In [1]:
from ultralytics import YOLO

print("\n--- Starting YOLO Training with Early Stopping ---")
model = YOLO('yolov8n.pt')

model.train(
    data="C:/Users/samue/MotorcycleDetection-2/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name='motorcycle-yolo',
    patience=10  # <-- stop training jika val loss tidak membaik selama 10 epoch
)



--- Starting YOLO Training with Early Stopping ---
Ultralytics 8.3.158  Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/samue/MotorcycleDetection-2/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=motorcycle-yolo12, nbs=64, nms=False, opset=None, optimize=

train: Scanning C:\Users\samue\MotorcycleDetection-2\train\labels.cache... 1052 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1052/1052 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 72.929.6 MB/s, size: 46.4 KB)


val: Scanning C:\Users\samue\MotorcycleDetection-2\val\labels.cache... 219 images, 0 backgrounds, 0 corrupt: 100%|██████████| 219/219 [00:00<?, ?it/s]


Plotting labels to runs\detect\motorcycle-yolo12\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\motorcycle-yolo12
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        219        219      0.676      0.333     0.0166     0.0133
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



11 epochs completed in 0.050 hours.
Optimizer stripped from runs\detect\motorcycle-yolo12\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\motorcycle-yolo12\weights\best.pt, 6.2MB

Validating runs\detect\motorcycle-yolo12\weights\best.pt...
Ultralytics 8.3.158  Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.89it/s]


                   all        219        219    0.00185      0.777     0.0783     0.0511
            motorBebek        110        110    0.00203      0.627    0.00171   0.000365
          motorListrik          1          1   0.000709          1    0.00716    0.00369
            motorMatic        108        108    0.00282      0.704      0.226      0.149
Speed: 0.4ms preprocess, 3.2ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs\detect\motorcycle-yolo12


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000253145CB8E0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [2]:
from ultralytics import YOLO
import os
import cv2
import matplotlib.pyplot as plt

# Load model terbaik hasil training
model = YOLO('runs/detect/motorcycle-yolo12/weights/best.pt')

# Path ke folder gambar yang ingin dideteksi
image_folder = "C:/Users/samue/MotorcycleDetection-2/test/images"
output_folder = "C:/Users/samue/MotorcycleDetection-2/test/predicted"
os.makedirs(output_folder, exist_ok=True)

# Loop semua file gambar
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)

    # Deteksi menggunakan YOLOv8
    results = model(image_path, save=False)

    # Simpan hasil deteksi (gambar dengan bounding box)
    annotated_frame = results[0].plot()
    save_path = os.path.join(output_folder, image_name)
    cv2.imwrite(save_path, annotated_frame)

    print(f"Saved: {save_path}")



image 1/1 C:\Users\samue\MotorcycleDetection-2\test\images\-Images_180_jpg.rf.c8c5f03593abc0096e1ccc3ea3e0c790.jpg: 640x640 (no detections), 16.2ms
Speed: 6.3ms preprocess, 16.2ms inference, 236.9ms postprocess per image at shape (1, 3, 640, 640)
Saved: C:/Users/samue/MotorcycleDetection-2/test/predicted\-Images_180_jpg.rf.c8c5f03593abc0096e1ccc3ea3e0c790.jpg

image 1/1 C:\Users\samue\MotorcycleDetection-2\test\images\download-1-_jpeg.rf.6721c78c9e013cee9a26ce7e4c308a90.jpg: 640x640 (no detections), 16.5ms
Speed: 3.3ms preprocess, 16.5ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)
Saved: C:/Users/samue/MotorcycleDetection-2/test/predicted\download-1-_jpeg.rf.6721c78c9e013cee9a26ce7e4c308a90.jpg

image 1/1 C:\Users\samue\MotorcycleDetection-2\test\images\download-1-_jpg.rf.ce72df9fdf74702e6b12b035a5d09787.jpg: 640x640 (no detections), 22.6ms
Speed: 4.5ms preprocess, 22.6ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)
Saved: C:/Users/samue/Motorc

In [6]:
# 6. (Opsional) Evaluasi dan Prediksi
metrics = model.val()  # evaluasi pada validation set


Ultralytics 8.3.158  Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 75.714.1 MB/s, size: 45.5 KB)


val: Scanning C:\Users\samue\MotorcycleDetection-2\val\labels.cache... 219 images, 0 backgrounds, 0 corrupt: 100%|██████████| 219/219 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:08<00:00,  1.59it/s]


                   all        219        219    0.00186       0.78     0.0783     0.0508
            motorBebek        110        110    0.00206      0.636    0.00172   0.000365
          motorListrik          1          1   0.000711          1    0.00716    0.00369
            motorMatic        108        108    0.00282      0.704      0.226      0.148
Speed: 1.5ms preprocess, 13.8ms inference, 0.0ms loss, 6.9ms postprocess per image
Results saved to runs\detect\val


In [16]:
save=True,  # default True
save_period=-1,  # hanya simpan model terbaik (tidak simpan tiap epoch)


In [5]:
from ultralytics import YOLO

model = YOLO("runs/detect/motorcycle-yolo12/weights/best.pt")  # Pastikan ini file yang kamu upload

# Tampilkan info tentang model
print("Label kelas:", model.names)
print("Jumlah kelas:", model.model.nc)
print("Struktur model:")
model.info()

Label kelas: {0: 'motorBebek', 1: 'motorListrik', 2: 'motorMatic', 3: 'motorSport', 4: 'motorTrail'}
Jumlah kelas: 5
Struktur model:
Model summary: 129 layers, 3,011,823 parameters, 0 gradients, 8.2 GFLOPs


(129, 3011823, 0, 8.1984512)